# Step 2: Cropping ArcticDEM tif Files to AOI

This script processes ArcticDEM Strip files by cropping them to a specified Area of Interest (AOI) defined in a shapefile.

---

### `crop_tifs_to_aoi` Function
- Accepts an input folder with `.tif` files, an AOI shapefile, and an output folder path
- Ensures output directory exists
- Loads the AOI shapefile
- Reprojects the AOI to match the raster CRS if needed

Loops Through All .tif Files
- Skips empty .tifs (size = 0 MB)
- Uses `rasterio.mask` to crop raster by AOI geometry
- Updates metadata and saves the cropped TIFF in the output folder

Finally, the script runs the function with these inputs:
- `input_folder`: Folder containing original ArcticDEM TIFFs
- `shapefile_path`: AOI shapefile path
- `output_folder`: Where cropped TIFFs are saved

In [ ]:
import os
import rasterio
from rasterio.errors import RasterioIOError
from rasterio.mask import mask
import geopandas as gpd
from pathlib import Path

In [ ]:
def crop_tifs_to_aoi(input_folder, shapefile_path, output_folder):
    """
    Crops all TIFF files in a folder to the area defined by an AOI shapefile.

    Parameters:
        input_folder (str): Path to the folder containing TIFF files.
        shapefile_path (str): Path to the AOI shapefile.
        output_folder (str): Path to the folder to save cropped TIFF files.
    """
    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Load the shapefile
    aoi = gpd.read_file(shapefile_path)

    # Check if AOI is in the correct CRS (Coordinate Reference System)
    with rasterio.open(next(Path(input_folder).glob("*.tif"))) as src:
        raster_crs = src.crs
    if aoi.crs != raster_crs:
        print(f"Reprojecting AOI from {aoi.crs} to {raster_crs}")
        aoi = aoi.to_crs(raster_crs)

    # Loop through all TIFF files in the input folder
    for tif_path in Path(input_folder).glob("*.tif"):
        if tif_path.stat().st_size > 0:  # Check if the file size is greater than 0MB
            try:
                print(f"Processing {tif_path}")
                with rasterio.open(tif_path) as src:
                    # Crop the raster with AOI
                    out_image, out_transform = mask(src, aoi.geometry, crop=True)
                    out_meta = src.meta.copy()

                    # Update metadata for the cropped image
                    out_meta.update({
                        "driver": "GTiff",
                        "height": out_image.shape[1],
                        "width": out_image.shape[2],
                        "transform": out_transform
                    })

                    # Save the cropped image
                    output_path = Path(output_folder) / tif_path.name
                    with rasterio.open(output_path, "w", **out_meta) as dest:
                        dest.write(out_image)
                print(f"Saved cropped image to {output_path}")
            except (RasterioIOError, ValueError) as e:
                print(f"Skipping {tif_path} due to error: {e}")
        else:
            print(f"Skipping {tif_path} as it is empty (size = 0MB)")

In [ ]:
input_folder = "1_download/"
shapefile_path = "aoi/aoi_collapse_basin.shp"
output_folder = "2_crop/"

crop_tifs_to_aoi(input_folder, shapefile_path, output_folder)